In [1]:
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import csv


In [2]:
def get_keyword(URL = 'https://fennougrica.kansalliskirjasto.fi/handle/10024/87318'):

    page = requests.get(URL)
    df_list = pd.read_html(page.text) # this parses all the tables in webpages to a list
    df = df_list[0]
    return(df[1][4].replace(' komin kieli; коми-зырянский язык', ''))

In [3]:
books = []

for file in Path("data").glob("*md"):
    
    book = {}
    
    connection = open(file, "r")
    
    book['id'] = str(file).replace('data/', '').replace('.md', '')
    book['text'] = connection.read()
    
    books.append(book)

In [4]:
metadata = {}
 
filename = "metadata.tsv"
  
with open(filename, 'r') as meta_tsv:
      
    for line in csv.DictReader(meta_tsv, delimiter="\t"):
        
        dictionary = dict(line)
        
        metadata[dictionary['handle'].replace("http://fennougrica.kansalliskirjasto.fi/handle/", '').replace('/', '_')] = dictionary

In [5]:
# books[30]

In [6]:
for book in books:
    
    try:

        book['metadata'] = metadata[book['id']]
        
    except:
        
        print(f"Check {book['id']}")

In [377]:
# import shutil

In [378]:
#for book in books:
    
#    filename = book['metadata']['handle'].replace('http://fennougrica.kansalliskirjasto.fi/handle/', '').replace('/', '_')
    
#    file = open(f"data/{filename}.md", "w")
        
#    text = ''

#    for line in book['text'].split("\n"):
        
#        if line.isupper():
            
#            text += f"## {line}\n"
        
#        else:
            
#            text += f"{line}\n"
            
#    file.write(text)
    
#    file.close()

In [379]:
# for book in books:
    
#    filename = book['metadata']['handle'].replace('http://fennougrica.kansalliskirjasto.fi/handle/', '').replace('/', '_')
    
#    shutil.copy(f"data2/{book['id']}.txt", f"data/{filename}.md")

In [387]:
for book in books:
    
    print(book['metadata']['handle'], book['metadata']['fulab_url'])

http://fennougrica.kansalliskirjasto.fi/handle/10024/87472 http://komikyv.org/kpv/content/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%B0-%D1%82%D0%B5%D1%85%D0%BD%D0%B8%D0%BA%D0%B0-%D0%BF%D0%B0%D1%81%D1%8C%D0%BA%D0%B0%D0%BB%D3%A7%D0%BC
http://fennougrica.kansalliskirjasto.fi/handle/10024/67937 http://komikyv.org/kpv/content/%D1%81%D3%A7%D1%81%D1%82%D3%A7%D0%BC-%D0%BE%D0%BB%D3%A7%D0%BC-%E2%80%94-%D0%B4%D0%B7%D0%BE%D0%BD%D1%8C%D0%B2%D0%B8%D0%B4%D0%B7%D0%B0-%D0%BE%D0%BB%D3%A7%D0%BC
http://fennougrica.kansalliskirjasto.fi/handle/10024/87417 http://komikyv.org/kpv/contents/korym-silosuytomyn-otsog
http://fennougrica.kansalliskirjasto.fi/handle/10024/87318 http://komikyv.org/kpv/contents/kydzi-vidzny-kagaos-3-arossyan-8-odz
http://fennougrica.kansalliskirjasto.fi/handle/10024/67927 http://komikyv.org/kpv/contents/skot-vidzan-otuvya-karta
http://fennougrica.kansalliskirjasto.fi/handle/10024/67339 http://komikyv.org/kpv/contents/vidzoy-vor-sotchomys
http://fennougrica.kansalliskirjasto.fi/handle/10024/87

In [380]:
for book in books:
    
    start_block = f"""---
title: "{book['metadata']['title_cyr']}"
output:
  html_document:
    toc: true
    toc_float: true
---
"""
    
    filename = book['metadata']['handle'].replace('http://fennougrica.kansalliskirjasto.fi/handle/', '').replace('/', '_')
    
    file = open(f"{filename}.Rmd", "w")
    
    description_block = f"""
    
## Cite as
    
```
@book{{{filename},
author={{{book['metadata']['author_cyr']}}},
year={{{book['metadata']['year']}}},
title={{{book['metadata']['title_cyr']}}},
url={{{book['metadata']['handle']}}},
note={{Scanned in the National Library of Finland's Fenno-Ugrica project. Processed and proofread by {book['metadata']['proofreader']}.}}
}}
```
"""
    
    text = ''

    for line in book['text'].split("\n"):
        
        if line.isupper() and "#" not in line:
            
            text += f"### {line.title()}\n"
        
        else:
            
            text += f"{line}\n"
        

    file.write(start_block)
    file.write(text)

    file.write(description_block)
    
    file.close()

In [381]:
# books

In [382]:
titles = []

for book in books:
    
    entry = {}
    
    filename = book['metadata']['handle'].replace('http://fennougrica.kansalliskirjasto.fi/handle/', '').replace('/', '_')
    
    entry['category'] = book['metadata']['categories_eng'] 
    entry['title'] = book['metadata']['title']
    entry['file'] = filename
    
    titles.append(entry)
    
#    print(cats[book['metadata']['categories_eng']])
    
#    print(cats[book['metadata']['categories_eng']])
#    cats[book['metadata']['categories_eng']] = cats[book['metadata']['categories_eng']].append(book['metadata']['title']) 
    
    
    

In [383]:
titles_sorted = sorted(titles, key = lambda i: i['category'])

In [384]:
# titles

In [385]:
yaml = open("_site.yml", "w")

yaml_start = """
name: "Written Komi Corpus: Fenno-Ugrica"
output_dir: "docs"
navbar:
  title: "Fenno-Ugrica Corpus"
  left:
    - text: "Home"
      href: index.html
"""

yaml.write(yaml_start)

current_category = "Agriculture"

s = "  "

yaml.write(f"{s}{s}- text: \"{current_category}\"\n")
yaml.write(f"{s}{s}{s}menu:\n")

for title in titles_sorted:
        
    if title['category'] == current_category:
        
        yaml.write(f"{s}{s}{s}{s}- text: \"{title['title']}\"\n")
        yaml.write(f"{s}{s}{s}{s}{s}href: {title['file']}.html\n")
    
    else:
        
        yaml.write(f"{s}{s}- text: \"{title['category']}\"\n")
        yaml.write(f"{s}{s}{s}menu:\n")
        yaml.write(f"{s}{s}{s}{s}- text: \"{title['title']}\"\n")
        yaml.write(f"{s}{s}{s}{s}{s}href: {title['file']}.html\n")
        current_category = title['category']
        
yaml_end = """    - text: "About"
      href: about.html
"""

yaml.write(yaml_end)

yaml.close()


## PROLATIVES

In [8]:
import string

In [9]:
prolatives = []
 
filename = "komi-prolatives.tsv"
  
with open(filename, 'r') as meta_tsv:
      
    for line in csv.DictReader(meta_tsv, delimiter="\t"):
        
        dictionary = dict(line)
        
        prolatives.append(dictionary)

In [27]:
def strip_text(text):
    
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation + '— '))
    
    return(text)

In [33]:
import re

In [43]:
for sentence in prolatives:
    
#    print(sentence['sentence'])
    
    pattern = '[А-ЯӦІЁ]{2,}'
          
    result = re.search(pattern, sentence['sentence'])
    
    print(result.span())

(101, 112)
(13, 18)
(92, 97)
(43, 50)
(76, 83)
(50, 59)
(53, 59)
(104, 113)
(103, 108)
(4, 9)
(4, 9)
(94, 105)
(138, 143)
(56, 65)
(26, 32)
(62, 67)
(46, 53)
(53, 59)
(21, 26)
(5, 17)
(18, 26)
(154, 160)
(5, 10)
(186, 191)
(63, 70)
(22, 28)
(161, 168)
(3, 8)
(88, 93)
(83, 88)
(100, 105)
(7, 10)
(96, 101)
(105, 111)
(31, 36)
(70, 75)
(63, 70)
(165, 170)
(39, 51)
(5, 14)
(38, 43)
(41, 45)
(0, 7)
(48, 55)
(51, 58)
(9, 16)
(13, 20)
(23, 30)
(93, 100)
(7, 14)
(47, 54)
(23, 30)
(57, 64)
(14, 21)
(93, 100)
(6, 13)
(67, 74)
(40, 47)
(34, 41)
(17, 24)
(66, 73)
(27, 34)
(159, 166)
(19, 31)
(7, 17)
(61, 66)
(8, 13)
(8, 15)
(77, 83)
(51, 56)
(37, 42)
(5, 14)
(40, 47)
(126, 134)
(51, 59)
(145, 153)
(155, 167)
(28, 33)
(44, 48)
(35, 40)
(51, 56)
(97, 103)
(32, 36)
(0, 6)
(0, 12)
(0, 7)
(32, 37)
(4, 9)
(10, 15)
(7, 17)
(98, 105)
(80, 86)
(4, 9)
(65, 73)
(110, 122)
(24, 32)
(97, 106)
(45, 50)
(52, 62)
(19, 33)
(72, 79)
(77, 86)
(59, 67)
(59, 65)
(77, 84)
(68, 74)
(62, 67)
(46, 58)
(36, 44)
(23, 29)
(8

In [26]:
# books[10]

In [25]:
file = open("prolative_ids.txt", "w")

for sentence in prolatives:
    
    status = False
    
    text = strip_text(sentence['sentence'])
    
    for book in books:
        
        if text in strip_text(book['text']):
            
#            print(book['metadata']['year'], sentence['sentence'])
            status = True
            book_id = book['id']
#            book_title = book['title_cyr']
        
    if status == False:
    
        # print(sentence['sentence'])
        
        file.write('\n')
    
    else:
        
        file.write(f"{book_id}\n")
        print(book_id)

file.close()

10024_67339
10024_87316
10024_67321
10024_67927
10024_67379
10024_67321
10024_87404
10024_87447
10024_87316
10024_87472
10024_87472
10024_87358
10024_67345
10024_67339
10024_67980
10024_87436
10024_87436
10024_67379
10024_67379
10024_67982
10024_67379
10024_67927
10024_87472
10024_87321
10024_67316
10024_67290
10024_67316
10024_87418
10024_87472
10024_67345
10024_87472
10024_67982
10024_67339
10024_87355
10024_67354
10024_67345
10024_67339
10024_67335
10024_87404
10024_87472
10024_67326
10024_87472
10024_67317
10024_67297
10024_67968
10024_67315
10024_67297
10024_67297
10024_87346
10024_67340
10024_67937
10024_67968
10024_67290
10024_67937
10024_67308
10024_67317
10024_67321
10024_67345
10024_87346
10024_87329
10024_67982
10024_87316
10024_67312
10024_67321
10024_67321
10024_87401
10024_87401
10024_67317
10024_67289
10024_87472
10024_67321
10024_87404
10024_67290
10024_87346
10024_67982
10024_67927
10024_67927
10024_67982
10024_87472
10024_87472
10024_87472
10024_87472
10024_87418
1002

## BIBTEX

In [293]:
bib = open("fennougrica.bib", "w")

nocite = []

for item in metadata:
    
    bib.write(f"""@book{{{item['id']},
author={{{item['author_cyr']}}},
year={{{item['year']}}},
title={{{item['title_cyr']}}},
url={{{item['handle']}}},
note={{Scanned in the National Library of Finland's Fenno-Ugrica project. Proofread by FU-Lab, Syktyvkar.}},
keywords={{fennougrica}}
}}

""")
    
    nocite.append(item['id'])

bib.close()


TypeError: string indices must be integers

In [ ]:
for cite in nocite:
    
    print(f"\\nocite{{{cite}}}")

In [ ]:
metadata